## Start

In [2]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)



___
### Tags: #Tables

### Links: 

[[deals_base_dir_bitrix]]

[[category_id_dir_bitrix]]

[[companies_id_dir_bitrix]]

[[companies_dir_partner]]
___

In [5]:
query_text = """--sql
WITH t1 AS(
SELECT 
	bdb.ID AS id,
	bdb.TITLE AS title,
	bdb.OPPORTUNITY AS opportunity ,
--	bdb.STAGE_ID AS stage_id ,
--	cidb.name AS stage_name,
--	bdb.CURRENCY_ID AS currency_id, 
	bdb.COMPANY_ID AS company_id,
	cidbc.company_name AS bitrix_company_name,
	toDate(parseDateTime32BestEffortOrNull(bdb.BEGINDATE)) AS date_start ,
	toDate(parseDateTime32BestEffortOrNull(bdb.CLOSEDATE)) AS date_finish,
	if(cidbc.partner_lk = 0,Null,cidbc.partner_lk) AS  partner_lk
FROM db1.deals_base_dir_bitrix_ch AS bdb 
LEFT JOIN (SELECT
			*
			FROM db1.category_id_dir_bitrix_ch
			WHERE category_id = 0
			)AS cidb ON cidb.stage_id = bdb.STAGE_ID
LEFT JOIN db1.companies_id_dir_bitrix_ch AS cidbc 
	ON toInt16(bdb.COMPANY_ID) = cidbc.company_id
WHERE bdb.CURRENCY_ID = 'RUB'
		AND stage_id = 'WON'
	)
--
SELECT
	t1.*,
	cdp.company_name AS partner_comapany_name
FROM t1
LEFT JOIN db1.companies_dir_partner  AS cdp 
	ON toInt16(t1.partner_lk) = toInt16(cdp.partner_lk)
ORDER BY date_finish DESC
"""

ch.get_schema(query_text)

(
    `id` UInt64,
    `title` String,
    `opportunity` Float64,
    `company_id` String,
    `bitrix_company_name` String,
    `date_start` Date,
    `date_finish` Date,
    `partner_lk` UInt32,
    `partner_comapany_name` String
)


In [6]:
query_text = """--sql
CREATE TABLE db1.t_bitrix_deals_sales
(
    `id` UInt64,
    `title` String,
    `opportunity` Float64,
    `company_id` String,
    `bitrix_company_name` String,
    `date_start` Date,
    `date_finish` Date,
    `partner_lk` UInt32,
    `partner_comapany_name` String
)
    ENGINE = MergeTree()
    ORDER BY id
"""

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [7]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.t_bitrix_deals_sales_mv
    REFRESH EVERY 1 DAY OFFSET 5 HOUR 35 MINUTE TO db1.t_bitrix_deals_sales AS
    WITH t1 AS(
    SELECT 
        bdb.ID AS id,
        bdb.TITLE AS title,
        bdb.OPPORTUNITY AS opportunity ,
    --	bdb.STAGE_ID AS stage_id ,
    --	cidb.name AS stage_name,
    --	bdb.CURRENCY_ID AS currency_id, 
        bdb.COMPANY_ID AS company_id,
        cidbc.company_name AS bitrix_company_name,
        toDate(parseDateTime32BestEffortOrNull(bdb.BEGINDATE)) AS date_start ,
        toDate(parseDateTime32BestEffortOrNull(bdb.CLOSEDATE)) AS date_finish,
        if(cidbc.partner_lk = 0,Null,cidbc.partner_lk) AS  partner_lk
    FROM db1.deals_base_dir_bitrix_ch AS bdb 
    LEFT JOIN (SELECT
                *
                FROM db1.category_id_dir_bitrix_ch
                WHERE category_id = 0
                )AS cidb ON cidb.stage_id = bdb.STAGE_ID
    LEFT JOIN db1.companies_id_dir_bitrix_ch AS cidbc 
        ON toInt16(bdb.COMPANY_ID) = cidbc.company_id
    WHERE bdb.CURRENCY_ID = 'RUB'
            AND stage_id = 'WON'
        )
    --
    SELECT
        t1.*,
        cdp.company_name AS partner_comapany_name
    FROM t1
    LEFT JOIN db1.companies_dir_partner  AS cdp 
        ON toInt16(t1.partner_lk) = toInt16(cdp.partner_lk)
    ORDER BY date_finish DESC
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

___
## Tools
___
### query


In [10]:
query_text = """--sql
    SELECT
        *
    FROM db1.t_bitrix_deals_sales
    limit 10
    """

ch.query_run(query_text)


id,title,opportunity,company_id,bitrix_company_name,date_start,date_finish,partner_lk,partner_comapany_name
i64,str,f64,str,str,date,date,i64,str
2289,"""Продажа 4 х стёкол для домофон…",0.0,"""3531""","""МЕТАКОМ СЕРВИС ООО НН""",2020-10-26,2020-10-31,120408,"""Метаком Сервис"""
2295,"""Заказ клиента 00ЦБ-00248""",9730.0,"""3523""","""СИТИЛАН ООО""",2020-10-16,2021-02-08,122029,"""ООО СИТИЛАН"""
2297,"""Заказ клиента 00ЦБ-00245""",105600.0,"""4183""","""КАСКАД ООО""",2020-10-14,2020-10-28,120653,"""ООО ""Каскад"""""
2299,"""Заказ клиента 00ЦБ-00249""",7900.0,"""4629""","""РИФТ ООО""",2020-10-16,2021-02-08,121950,"""ООО «Рифт»"""
2301,"""Заказ клиента 00ЦБ-00250""",35200.0,"""4631""","""Гончаров Вячеслав Вячеславович""",2020-10-16,2020-10-28,124145,"""Индивидуальный предприниматель…"
2303,"""Заказ клиента 00ЦБ-00238""",422400.0,"""4281""","""АСКО СЕРВИС ООО""",2020-10-13,2020-10-18,120202,"""Aско Сервис ООО ПРО"""
2305,"""Заказ клиента 00ЦБ-00251""",35200.0,"""4633""","""ДОЗОР ООО""",2020-10-16,2020-10-28,0,""""""
2307,"""Заказ клиента 00ЦБ-00252""",7900.0,"""4639""","""ОБНИНСК - ТЕЛЕКОМ ООО""",2020-10-16,2021-02-08,122040,"""ООО ""СКАУТ"""""
2309,"""Заказ клиента 00ЦБ-00244""",35200.0,"""4623""","""МОДУС ООО""",2020-10-14,2020-10-28,123194,"""Модус"""


### delete a part


In [ ]:
query_text = """--sql
    ALTER TABLE db1.t_bitrix_deals_sales DELETE WHERE report_date = '2025-07-17'
    """

ch.query_run(query_text)


### drop mv

In [ ]:
query_text = """--sql
    DROP TABLE db1.t_bitrix_deals_sales_mv
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### drop ch

In [ ]:
query_text = """--sql
    DROP TABLE db1.t_bitrix_deals_sales
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### refresh mv

In [ ]:
query_text = """
SYSTEM REFRESH VIEW db1.t_uk_adresses_subscribtions_mv
"""

ch.query_run(query_text)

elapsed_ns,query_id
i64,str
897986,"""58d516b8-81ce-47ef-ab2c-c0196a…"
